<!--Document description-->

*This document details the parameterization of COBALT for the Kaneohe (hioekg) grid.

*This is a live document that will be updated as needed.*

*Last update:* 2019-01-31

# Parameterizing COBALT

## Create NetCDF files to run COBALT

1. Navigate to ``/nest``.


2. **setup_inner_frc.py**
    * Chunk ``#1``: Sets up forcing from rivers, atmosphere; loads from internet. Note that if I run this and create a forcing file, but then need to change something, I need to delete that previously created file. otherwise, the script will “see” that file exists and not recreate it.
    * Chunk ``#2``: Creates **frc-cobalt.nc**.
    * Chunk ``#3``: Creates **river-hioekg.nc**. It may be helpful to look at documentation of ``psource.create``.
            i. I can change what I specify under ``seapy.roms.cobalt.add_psource_var(…``.
            ii. Temperature doesn’t really matter.
            iii. We always want salinity to = 5; it's a good approximation right now.
    * To run:
```python
qexec python setup_inner_frc.py
```


3. **setup_inner_nest.py**
    * Here, change only the start day, end day, and pathway (maybe) to outer grid.
    * To run:
```python
qexec -P4 python setup_inner_nest.py
```